In [24]:
from context import opencaching, openroute, matrix_utils

def print_response(response: dict):
    matrix_utils.print_matrix(response['distances'], "Distances")
    print('\n')
    matrix_utils.print_matrix(response['durations'], "Durations")

In [25]:
openroute.api.go_offline()
local_ready = openroute.check_status()
print("[{}] localhost".format(' OK ' if local_ready else 'FAIL'))

openroute.api.go_online()
azure_ready = openroute.check_status()
print("[{}] azure".format(' OK ' if azure_ready else 'FAIL'))

openroute.api.toggle_online(not local_ready)

print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))

[FAIL] localhost
[ OK ] azure
Using: azure


In [3]:
start_coord = (52.518848, 13.399411)

In [4]:
import json

start_lat, start_lon = start_coord
geocaches = opencaching.parse_results(opencaching.search(start_lat, start_lon, count=500))

locations_latlon = [(cache['latitude'], cache['longitude']) for cache in geocaches]
locations_lonlat = [(cache['longitude'], cache['latitude']) for cache in geocaches]

with open("data/geocaches.json", 'w+') as outfile:
    json.dump({
        "geocaches": geocaches,
        "locations_latlon": locations_latlon,
        "locations_lonlat": locations_lonlat
    }, outfile)

print(f"Found {len(geocaches)} geocaches near {start_coord}")

Found 500 geocaches near (52.518848, 13.399411)


In [5]:
import folium

def create_marker(cache: dict, color: str = 'darkred'):
    return folium.map.Marker(
        location=[cache['latitude'], cache['longitude']],
        popup=f"<strong>{cache['code']}</strong><br>Lat:&nbsp;{cache['latitude']:.3f}<br>Lon:&nbsp;{cache['longitude']:.3f}",
        icon=folium.map.Icon(color=color), tooltip=cache['code']
    )

def fit_locations(map: folium.Map, locations: list):
    sw = min(locations)
    ne = max(locations)
    map.fit_bounds([sw, ne])

m = folium.Map(tiles='Stamen Toner',location=start_coord, zoom_start=11)

In [6]:
for cache in geocaches[:10]:
    create_marker(cache, 'darkblue').add_to(m)
fit_locations(m, locations_latlon[:10])
m

In [7]:
matrix_10x10 = openroute.request_matrix({
    "locations": locations_lonlat[:10],
    "metrics": ["distance", "duration"]
}, nocache=True)

In [8]:
print_response(matrix_10x10)

Distances (10x10 matrix):
[[   0.    613.84  721.86  556.77  876.35  959.67  984.21 1161.4   877.96
  1006.38]
 [ 613.84    0.    313.8   901.14  885.11 1358.21  690.33 1625.94 1080.09
  1052.39]
 [ 719.45  313.8     0.   1295.78 1012.44 1555.62  443.87 1753.26 1474.74
  1198.32]
 [ 574.29  918.66 1202.53    0.   1422.86  746.53 1396.41 1051.39  302.16
  1019.08]
 [ 794.13  847.11  974.43 1233.9     0.   1474.78 1489.5  1489.67 1672.08
  1800.5 ]
 [ 924.47 1327.52 1550.17  692.35 1539.39    0.   1697.9   462.86  642.73
  1387.81]
 [ 852.99  608.95  389.75 1275.53 1526.12 1597.37    0.   1817.31 1338.11
   898.98]
 [1109.34 1628.13 1755.45 1008.26 1503.67  462.86 1922.57    0.   1184.13
  1703.72]
 [ 844.96 1179.17 1463.03  302.16 1721.31  777.79 1443.83 1173.08    0.
   773.62]
 [ 995.14 1052.39 1215.99 1047.1  1871.49 1493.99  916.65 1798.85  773.62
     0.  ]]


Durations (10x10 matrix):
[[   0.    441.95  519.73  400.86  630.96  690.94  708.62  836.19  632.11
   724.57]
 [ 441.95   

In [9]:
import json

with open("data/matrix10.json", 'w+') as outfile:
    json.dump(matrix_10x10, outfile)

In [10]:
for cache in geocaches[10:50]:
    create_marker(cache, 'darkgreen').add_to(m)
fit_locations(m, locations_latlon[:50])
m

In [11]:
# matrix_50x50 = openroute.matrix_builder(locations_lonlat[:50], nocache=True)

In [12]:
# print_response(matrix_50x50)

In [13]:
# with open("data/matrix50.json", 'w+') as outfile:
#     json.dump(matrix_50x50, outfile)

In [14]:
# for cache in geocaches[50:500]:
#     create_marker(cache).add_to(m)
# m

In [15]:
# matrix_500x500 = openroute.matrix_builder(locations_lonlat, increments=20, nocache=True)

In [16]:
# print_response(matrix_500x500)

In [17]:
# with open("data/matrix500.json", 'w+') as outfile:
#     json.dump(matrix_500x500, outfile)

In [22]:
first_id = 131
second_id = 132

matrix_n0_n94 = openroute.request_matrix({
    "locations": [locations_lonlat[first_id], locations_lonlat[second_id]],
    "metrics": ["distance", "duration"]
}, nocache=True)

print_response(matrix_n0_n94)

print([locations_lonlat[first_id], locations_lonlat[second_id]])

Distances (2x2 matrix):
[[None None]
 [None None]]


Durations (2x2 matrix):
[[None None]
 [None None]]
[(13.325933, 52.4812), (13.325933, 52.4812)]


In [19]:
import folium
from algo_results import create_marker, create_circle_marker

In [21]:
from context import openroute

m = folium.Map(tiles=None, location=locations_latlon[0], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(m)

def style_function(color, weight=3, opacity=1):
    return lambda feature: dict(color=color, weight=weight, opacity=opacity)

create_circle_marker(locations_latlon[first_id], color='blue', radius=8).add_to(m)
create_circle_marker(locations_latlon[second_id], color='red', radius=8).add_to(m)

m.fit_bounds([locations_latlon[first_id], locations_latlon[second_id]])

request = {
    'coordinates': [locations_lonlat[first_id], locations_lonlat[second_id]],
    'geometry': True,
    'instructions': False
}
route_n0_n94 = openroute.request_directions(request, format_out='/geojson')
folium.features.GeoJson(data=route_n0_n94,
                        name=f'{first_id} -> {second_id}',
                        style_function=style_function('blue', 4),
                        overlay=True).add_to(m)

request['coordinates'] = [locations_lonlat[second_id], locations_lonlat[first_id]]
route_n94_n0 = openroute.request_directions(request, format_out='/geojson')
folium.features.GeoJson(data=route_n94_n0,
                        name=f'{second_id} -> {first_id}',
                        style_function=style_function('red', 2),
                        overlay=True).add_to(m)

m